In [22]:
#! pip install seqeval
#! pip install evaluate
#! pip install pandas
#! pip install datasets
#! pip install torch
#! pip install transformers
#! pip install scikit-learn
#! pip install ninja
#! pip install flash-attn
#! pip install packaging
#! pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\magnu\\AppData\\Local\\Temp\\pip-install-_h0lj359\\flash-attn_a64e364bab4b4bda827435c2541f476a\\csrc\\composable_kernel\\library\\include\\ck\\library\\tensor_operation_instance\\gpu\\grouped_conv_bwd_weight\\device_grouped_conv_bwd_weight_two_stage_xdl_instance.hpp'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import json
import torch
#import evaluate
#import seqeval
#import accelerate
import transformers
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split

#from flash_attn.flash_attention import FlashAttention
# Flash Attention for faster training

torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32 for better performance
torch.backends.cudnn.benchmark = True  # Enable CuDNN auto-tuner

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Load dataset
DATA_PATH = "tokenized_ner_data_4.json"
with open(DATA_PATH, "r") as f:
    data = json.load(f)

In [7]:
# Extract all unique label types from dataset
unique_labels = list(set(label for entry in data for label in entry["labels"]))

# Create mapping from label name -> index
label_to_id = {label: i for i, label in enumerate(unique_labels)}

# Convert dataset labels from strings to integer IDs
for entry in data:
    entry["labels"] = [label_to_id[label] for label in entry["labels"]]

In [10]:
def preprocess_data(data):
    tokenized_inputs = []
    tokenized_labels = []

    for entry in data:
        tokens = entry["tokens"]
        labels = entry["labels"]

        tokenized_inputs.append(tokens)
        tokenized_labels.append(labels)
    return {"tokens": tokenized_inputs, "labels": tokenized_labels}

dataset = Dataset.from_dict(preprocess_data(data))
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()

In [11]:
# Load tokenizer and model
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(set([label for entry in data for label in entry["labels"]]))).to(device)

# Enable Flash Attention in the model (if applicable)
#model.config.use_flash_attention = True

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

C:\Users\magnu\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\magnu\.cache\huggingface\hub\models--answerdotai--ModernBERT-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [35]:
# Process dataset with tokenizer and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        #padding="max_length",
        #max_length=512
    )
    labels = []
    
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get mapping of subwords to words
        previous_word_id = None
        label_ids = []
        
        for word_id in word_ids:
            if word_id is None:  # Ignore special tokens (CLS, SEP, PAD)
                label_ids.append(-100)
            elif word_id != previous_word_id:  # Assign correct label only to first subword
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)  # Assign -100 to subsequent subwords
            
            previous_word_id = word_id

        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    tokenized_inputs["decoded"] = [[tokenizer.decode(i) for i in ids] for ids in tokenized_inputs.input_ids]

    # 🔹 Debugging Step: Print a Sample to Verify Alignment
    print("\n==== Sample Debugging ====")
    print("Len examples: ", len(examples["tokens"][-1]))
    print("Len tokenized inputs: ", len(tokenized_inputs["input_ids"][-1]))
    print("Original Tokens: ", examples["tokens"][-1])
    print("decoded: ", tokenized_inputs["decoded"][-1])
    print("Tokenized Input IDs: ", tokenized_inputs["input_ids"][-1])
    print("Word IDs Mapping: ", word_ids)
    print("Aligned Labels: ", labels[-1])
    print("==========================\n")
    
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset format
dataset = Dataset.from_dict({"tokens": [entry["tokens"] for entry in data], "labels": [entry["labels"] for entry in data]})

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]


==== Sample Debugging ====
Len examples:  512
Len tokenized inputs:  692
Original Tokens:  ['[CLS]', 'Nuclear', 'waste', 'processing', 'facility', 'Se', '##lla', '##field', 'has', 'been', 'fined', '£', '##33', '##2', ',', '500', '(', '$', '440', '##k', ')', 'by', 'the', 'Office', 'for', 'Nuclear', 'Regulation', '(', 'ON', '##R', ')', 'for', 'failing', 'to', 'ad', '##here', 'to', 'c', '##y', '##bers', '##ec', '##urity', 'standards', 'and', 'putting', 'sensitive', 'nuclear', 'information', 'at', 'risk', 'over', 'four', 'years', ',', 'from', '2019', 'to', '202', '##3', '.', 'According', 'to', 'the', 'ON', '##R', 'announcement', ',', 'Se', '##lla', '##field', 'failed', 'to', 'follow', 'its', 'own', 'approved', 'c', '##y', '##bers', '##ec', '##urity', 'protocols', 'by', 'leaving', 'multiple', 'v', '##ul', '##ner', '##abi', '##lities', 'in', 'its', 'IT', 'systems', 'un', '##pa', '##tched', ',', 'violating', 'the', 'Nuclear', 'Industries', 'Security', 'Regulations', '2003', '.', 'Although', 

In [30]:
tokenizer([" Nuclear", "waste"], add_special_tokens=False)

{'input_ids': [[27456], [4238, 442]], 'attention_mask': [[1], [1, 1]]}

In [20]:
# Load metric
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[label for label in label_seq if label != -100] for label_seq in labels]
    true_predictions = [[pred for pred, lab in zip(pred_seq, label_seq) if lab != -100] for pred_seq, label_seq in zip(predictions, labels)]
    return seqeval.compute(predictions=true_predictions, references=true_labels)

In [21]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",     # Output directory
    eval_strategy="epoch", # Evaluate after each epoch
    save_strategy="epoch",      # Save model after each epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",       # Log directory
    logging_steps=10,
    save_total_limit=2,         # Save last 2 models only
    fp16=True,                  # Enable mixed precision training for better performance
    bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if supported for better speed
    optim="adamw_torch_fused",  # Use fused optimizer for better CUDA performance
)

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [22]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


C:\Users\magnu\AppData\Local\Temp\ipykernel_10116\695041789.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./ner_model")
tokenizer.save_pretrained("./ner_model")

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []